In [22]:
import psycopg2
import pandas as pd

# The connection details (user, password, host) below are placeholders.
# Your coach will provide you with the actual connection details for your exercises.

conn_params = {
    'host': 'etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com',
    'dbname': 'postgres',
    'user': 'readonly_user',
    'password': 'you-can-only-read'
}

In [23]:
def fetch_data(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames

In [24]:
# Extract bookings data
query = "SELECT facid, slots FROM bookings;"
bookings_data, colnames = fetch_data(query)
bookings_df = pd.DataFrame(bookings_data, columns=colnames)

# Transform data: Calculate total booking duration per facility
# Assuming each slot represents 30 minutes of booking time
bookings_df['booking_duration'] = bookings_df['slots'] * 30  # Convert slots to minutes
total_duration_per_facility = bookings_df.groupby('facid')['booking_duration'].sum().reset_index()
total_duration_per_facility.columns = ['FacilityID', 'TotalBookingDurationInMinutes']

In [25]:
etl_bites_conn_string = "host='localhost' port='5432' dbname='etl_bites' user='jackdench' "

In [26]:
def execute_query_postgresql(conn_string, query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

# Define the connection string for your analytical database
etl_bites_conn_string = "dbname='etl_bites' user='jackdench' host='localhost' port='5432'"

# SQL query to create a new table for storing total booking duration per facility
create_total_duration_table = '''
CREATE TABLE total_booking_duration (
    FacilityID INTEGER NOT NULL,
    TotalBookingDurationInMinutes INTEGER NOT NULL
);
'''

# Execute the query to create the table
execute_query_postgresql(etl_bites_conn_string, create_total_duration_table)

In [27]:
def insert_data(conn_string, table_name, data, columns):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

# Insert the transformed data into the analytical database
insert_data(etl_bites_conn_string, 'total_booking_duration', total_duration_per_facility, ['FacilityID', 'TotalBookingDurationInMinutes'])